In [1]:
# importamos las librerías necesarias
import numpy as np
import math as m
import sympy as sp
from sympy import *



In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)
print("hola")

hola


<h3>CONNECT TO COPPELIASIM</h3>

In [3]:
import sim
import time
def connect(port):
# Establece la conexión a COPPELIA
# El port debe coincidir con el puerto de conexión en VREP  -- DALE AL PLAY !!!
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

In [4]:
# Conectarse al servidor de COPPELIA
# *** _Hay que ejecutarlo cada vez que se reinicia la simulación ***
clientID = connect(19999)

conectado a 19999


<h3>HANDLE INITIALIZATION</h3>

In [5]:
#==================== OBJECTS ====================
#ref_point for delta Kinematics
retCode,target=sim.simxGetObjectHandle(clientID,'ref_point',sim.simx_opmode_blocking)
#home for delta kinematics relative to traslation movement
retCode,home=sim.simxGetObjectHandle(clientID,'home',sim.simx_opmode_blocking)

#==================== JOINTS ====================
#Delta rotation joint
retcode,rotate_joint = sim.simxGetObjectHandle(clientID,'Rotate_joint',sim.simx_opmode_blocking)
#delta traslation joint
retcode,traslate_joint = sim.simxGetObjectHandle(clientID,'Traslate_joint',sim.simx_opmode_blocking)
desplazamiento = 0
#gripper revolute joint
retcode, grevolute_joint = sim.simxGetObjectHandle(clientID,'GripperRevolute_joint',sim.simx_opmode_blocking)

#==================== SENSORS ====================
#Gripper sensor
retcode,gripper_sensor = sim.simxGetObjectHandle(clientID,'Proximity_sensor',sim.simx_opmode_blocking)
#Camera
retcode,camera_sensor = sim.simxGetObjectHandle(clientID,'DVS128_sensor',sim.simx_opmode_blocking)

#==================== OTHER ====================
#Ground anchor attachpoint
retcode,ground_attachpoint = sim.simxGetObjectHandle(clientID,'anchor_attachpoint',sim.simx_opmode_blocking)
#Gripper attachpoint
retcode,gripper_attachpoint = sim.simxGetObjectHandle(clientID,'ROBOTIQ_85_attachPoint',sim.simx_opmode_blocking)


In [6]:
retCode,posicion=sim.simxGetObjectPosition(clientID,target,home,sim.simx_opmode_blocking)
posicion

[6.05359673500061e-07, 0.024999620392918587, -7.152557373046875e-07]

<h3>HANDLE INITIALIZATION</h3>

In [7]:
#Funcion para abrir o cerrar la pinza remotamente:
def gripper(val, parent, child):
    # función que acciona el efector final remotamente
    #val es Int con valor 0 ó 1 para desactivar o activar el actuador final. res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
    "ROBOTIQ_85", sim.sim_scripttype_childscript,"gripperLUA",[val,parent,child],[],[],"", sim.simx_opmode_blocking)
    return res
#Funcion para detectar objecto lego
def detect(sensor):
    errorCode, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector=sim.simxReadProximitySensor(clientID,sensor, sim.simx_opmode_blocking)
    # podemos calcular la distancia con  #simx_opmode_buffer
    if detectionState==True:
        sensor_val=np.linalg.norm(detectedPoint)
        print("handle objeto detectado: ", detectedObjectHandle)
        print("distacia al objeto: ",sensor_val)
        return [detectedObjectHandle, sensor_val]
    else:
        print("No ha detectado objeto")
    return 0
    


In [8]:
#sim.simxSetObjectPosition(clientID,target,-1,posicion_home,sim.simx_opmode_oneshot)

In [9]:

retCode,posicion_home=sim.simxGetObjectPosition(clientID,home, -1, sim.simx_opmode_blocking)
posicion_home
    

[-0.0006956029683351517, 0.0004104080726392567, 0.8643853664398193]

<h3> Number one </h3>

<h4>First piece</h4>

In [10]:
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
    
#go to beggining of row
for i in range(0, 500, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

#go back to center
for i in range(500, 0, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        

movetoPlaceZone = 0
for i in range(85,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)


#place
for i in range(0,-750,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-750,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

time.sleep(1)

handle objeto detectado:  158
distacia al objeto:  0.11359205842090803


<h4>Second piece</h4>

In [11]:
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

#go to next row for different piece
for i in range(0, -250, -5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#go to beggining of row
for i in range(0, 500, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

    
    
#go back to center

for i in range(500, 0, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
for i in range(-250,0,5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)        

movetoPlaceZone = 0
for i in range(85,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)


#place
for i in range(0,-630,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-630,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        
time.sleep(1)

handle objeto detectado:  185
distacia al objeto:  0.11359224468542295


<h4>Third piece</h4>

In [12]:
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

#go to next row for different piece
for i in range(0, -250, -5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#go to next piece in row
for i in range(0, 360, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

    
    
#go back to center

for i in range(360, 0, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
for i in range(-250,0,5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)        

movetoPlaceZone = 0
for i in range(85,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)


#place
for i in range(0,-510,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-510,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        
time.sleep(1)

handle objeto detectado:  187
distacia al objeto:  0.11359230429006772


<h4>Fourth piece</h4>

In [13]:
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
    
#go to piece
for i in range(0, 360, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

#go back to center
for i in range(360, 0, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        

movetoPlaceZone = 0
for i in range(85,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
time.sleep(1)
#correct x 
for i in range(0,-70,-5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005) 

#place
for i in range(0,-395,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-395,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#correct x back to center
for i in range(-70,0,5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005) 

handle objeto detectado:  159
distacia al objeto:  0.11359202116800504


<h4> Fifth piece <h/4>

In [14]:
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

#go to next row for different piece
for i in range(0, -250, -5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#go to next piece in row
for i in range(0, 230, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    


for i in range(230, 0, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
for i in range(-250,0,5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)        

movetoPlaceZone = 0
for i in range(85,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)


#place
for i in range(0,-280,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-280,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        
time.sleep(1)

handle objeto detectado:  189
distacia al objeto:  0.11359224468536473


<h3> Number two </h3>

<h4>First piece</h4>

In [15]:
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
    
#go to beggining of row
for i in range(0, 220, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

#go back to center
for i in range(220, 0, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        

movetoPlaceZone = 0
for i in range(85,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

time.sleep(1)
movetoPlace2Zone = 0.40
for i in range(0,-40,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

#place
for i in range(0,-750,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-750,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

time.sleep(1)

handle objeto detectado:  160
distacia al objeto:  0.1160923764117039


<h4>Second piece</h4>

In [16]:

movetoPlace2Zone = 0.40
for i in range(-40,0,1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
time.sleep(1)
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

#go to next row for different piece
for i in range(0, -250, -5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#go to beggining of row
for i in range(0, 73, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

#go back to center

for i in range(73, 0, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
for i in range(-250,0,5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)        

movetoPlaceZone = 0
for i in range(80,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
time.sleep(1)
movetoPlace2Zone = 0.40
for i in range(0,-40,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
    
#correct x 
for i in range(0,-50,-5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005) 
    
#place
for i in range(0,-630,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-630,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        
#correct x back to center
for i in range(-50,0,5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005) 

time.sleep(1)

handle objeto detectado:  191
distacia al objeto:  0.11359230429006772


<h4>Third piece</h4>

In [17]:
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
    
#go to beggining of row
for i in range(0, 73, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

#go back to center
for i in range(73, 0, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        

movetoPlaceZone = 0
for i in range(85,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

time.sleep(1)
movetoPlace2Zone = 0.40
for i in range(0,-40,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

#place
for i in range(0,-510,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-510,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        
time.sleep(1)

handle objeto detectado:  169
distacia al objeto:  0.11358352005554391


<h4>Fourth piece</h4>

In [18]:

movetoPlace2Zone = 0.40
for i in range(-40,0,1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
time.sleep(1)
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

#go to next row for different piece
for i in range(0, -250, -5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#go to beggining of row
for i in range(0, -73, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

#go back to center

for i in range(-73, 0, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
for i in range(-250,0,5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)        

movetoPlaceZone = 0
for i in range(80,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
time.sleep(1)
movetoPlace2Zone = 0.40
for i in range(0,-40,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
    
#correct x 
for i in range(0,60,5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005) 
    
#place
for i in range(0,-395,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-395,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        
#correct x back to center
for i in range(60,0,-5):
    #----X--------
    posicion[0] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005) 

time.sleep(1)

handle objeto detectado:  193
distacia al objeto:  0.11609259247854121


<h4>Fifth piece</h4>

In [19]:
movetoPickZone = -0.85
for i in range(0,-85,-1):
    aux = i/100
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)
    
#go to beggining of row
for i in range(0, -73, -5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
    
#pick
for i in range(0,-775,-5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

dres = detect(gripper_sensor)
holding = int(dres[0])
res = gripper(1, gripper_attachpoint, int(dres[0]))
time.sleep(1)
    
for i in range(-775,0,5):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

#go back to center
for i in range(-73, 0, 5):
    #----y--------
    posicion[1] = i/1000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        

movetoPlaceZone = 0
for i in range(85,0,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

time.sleep(1)
movetoPlace2Zone = 0.40
for i in range(0,-40,-1):
    aux = -(i/100)
    retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, aux, sim.simx_opmode_oneshot)
    time.sleep(0.00005)

#place
for i in range(0,-280,-10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)

res = gripper(0, gripper_attachpoint,holding)
holding = ""

time.sleep(1)
    
for i in range(-280,0,10):
    #----Z--------
    posicion[2] = i/2000
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.000005)
        
time.sleep(1)

handle objeto detectado:  161
distacia al objeto:  0.11359228938890653
